# Advanced Modeling: LSTM, LightGBM, CatBoost, Prophet, CNN-LSTM, Transformer

## 1. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import sys
import os

from sklearn.model_selection import TimeSeriesSplit 
from sklearn.metrics import classification_report, accuracy_score, roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src import config
from src.data_management import download_stock_data
from src.feature_engineering import add_technical_indicators, add_rolling_lag_features, create_target_variable
from src.modeling import (
    train_lstm, predict_lstm, 
    train_lightgbm, predict_lightgbm,
    train_catboost, predict_catboost,
    train_prophet, predict_prophet,
    train_cnnlstm, predict_cnnlstm, # Added CNN-LSTM
    train_transformer, predict_transformer # Added Transformer
)
from src.utils import save_model, load_model

## 2. Configuration and Parameters

In [ ]:
TICKER = config.DEFAULT_TICKERS[0] if config.DEFAULT_TICKERS else 'AAPL'
START_DATE = '2018-01-01' 
END_DATE = config.DEFAULT_END_DATE
INTERVAL = '1d'

FUTURE_DAYS_TARGET = 5
PERCENT_CHANGE_THRESHOLD = 0.03

ROLLING_WINDOWS = [5, 10, 20, 60]
LAG_PERIODS = [1, 2, 3, 5, 10]
KEY_LAG_INDICATORS = ['RSI_14', 'MACD', 'ATR_14', 'Stoch_k', 'ADX_14']

SEQUENCE_LENGTH = 20 
N_SPLITS_TIMESERIES = 5

FIT_PARAMS_DEFAULT = {'epochs': 50, 'batch_size': 32, 'validation_split': 0.1, 'verbose': 0}

LSTM_TRAIN_PARAMS = {
    'lstm_params': {'lstm_units': 50, 'dropout_rate': 0.2, 'dense_units_factor': 0.5},
    'fit_params': FIT_PARAMS_DEFAULT
}
CNNLSTM_TRAIN_PARAMS = {
    'cnnlstm_params': {'filters': 64, 'kernel_size': 3, 'pool_size': 2, 'lstm_units': 50, 'dropout_rate': 0.2, 'dense_units_factor': 0.5},
    'fit_params': FIT_PARAMS_DEFAULT
}
TRANSFORMER_TRAIN_PARAMS = {
    'transformer_params': {'embed_dim': 64, 'num_heads': 2, 'ff_dim': 32, 'num_transformer_blocks': 1, 'dropout_rate': 0.1},
    'fit_params': FIT_PARAMS_DEFAULT
}

LGBM_PARAMS = {
    'objective': 'binary', 'metric': 'binary_logloss', 'n_estimators': 200,
    'learning_rate': 0.05, 'num_leaves': 31, 'max_depth': -1,
    'feature_fraction': 0.8, 'bagging_fraction': 0.8, 'bagging_freq': 5, 'verbose': -1
}
CATBOOST_PARAMS = {
    'iterations': 200, 'learning_rate': 0.05, 'depth': 6,
    'loss_function': 'Logloss', 'eval_metric': 'AUC', 'verbose': 0,
    'early_stopping_rounds': 10
}
PROPHET_FORECAST_PERIOD = 90 

LSTM_MODEL_FILENAME = f"lstm_model_{TICKER.lower()}.h5"
CNNLSTM_MODEL_FILENAME = f"cnnlstm_model_{TICKER.lower()}.h5"
TRANSFORMER_MODEL_FILENAME = f"transformer_model_{TICKER.lower()}.h5"
LGBM_MODEL_FILENAME = f"lgbm_model_{TICKER.lower()}.txt"
CATBOOST_MODEL_FILENAME = f"catboost_model_{TICKER.lower()}.cbm"
PROPHET_MODEL_FILENAME = f"prophet_model_{TICKER.lower()}.json"

## 3. Load and Engineer Features

In [ ]:
raw_data_dl = download_stock_data([TICKER], START_DATE, END_DATE, INTERVAL)
if raw_data_dl is None: raise ValueError(f"Failed to download data for {TICKER}.")
processed_df = raw_data_dl.droplevel(0, axis=1) if isinstance(raw_data_dl.columns, pd.MultiIndex) else raw_data_dl.copy()

data_with_ti = add_technical_indicators(processed_df, fillna=True)
data_with_roll_lag = add_rolling_lag_features(data_with_ti, windows=ROLLING_WINDOWS, lags=LAG_PERIODS, lag_indicators=KEY_LAG_INDICATORS)
final_processed_data = create_target_variable(data_with_roll_lag, future_days=FUTURE_DAYS_TARGET, percent_change_threshold=PERCENT_CHANGE_THRESHOLD)
print(f"Final processed data shape: {final_processed_data.shape}")

base_cols = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
feature_cols = [col for col in final_processed_data.columns if col not in base_cols + ['target']]
X_unproc = final_processed_data[feature_cols]
y_unproc = final_processed_data['target']
ohlcv_unproc = final_processed_data[['Open', 'High', 'Low', 'Close', 'Volume']].copy()

model_input_data = X_unproc.assign(target=y_unproc)
cleaned_indices = model_input_data.dropna().index
X_full_cleaned = X_unproc.loc[cleaned_indices]
y_full_cleaned = y_unproc.loc[cleaned_indices]
ohlcv_full_cleaned = ohlcv_unproc.loc[cleaned_indices]
print(f"Shape after NaN drop: X: {X_full_cleaned.shape}, y: {y_full_cleaned.shape}, OHLCV: {ohlcv_full_cleaned.shape}")
if X_full_cleaned.empty: raise ValueError("No data after NaN drop.")

## 4. Split Data for Time Series Evaluation

In [ ]:
tscv = TimeSeriesSplit(n_splits=N_SPLITS_TIMESERIES)
train_idx, test_idx = None, None
for i, (idx_train, idx_test) in enumerate(tscv.split(X_full_cleaned)):
    if i == N_SPLITS_TIMESERIES - 1: train_idx, test_idx = idx_train, idx_test

X_train, X_test = X_full_cleaned.iloc[train_idx], X_full_cleaned.iloc[test_idx]
y_train, y_test = y_full_cleaned.iloc[train_idx], y_full_cleaned.iloc[test_idx]
ohlcv_train, ohlcv_test = ohlcv_full_cleaned.iloc[train_idx], ohlcv_full_cleaned.iloc[test_idx]

print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")

## 5. LSTM Model

In [ ]:
print("\n--- Training LSTM Model ---")
try:
    lstm_model, lstm_scaler = train_lstm(
        X_train_df=X_train.copy(), y_train_series=y_train.copy(),
        sequence_length=SEQUENCE_LENGTH, **LSTM_TRAIN_PARAMS
    )
    save_model(lstm_model, LSTM_MODEL_FILENAME, models_dir=config.MODELS_DIR)
    save_model(lstm_scaler, f"lstm_scaler_{TICKER.lower()}.pkl", models_dir=config.MODELS_DIR)
    y_pred_lstm_c, y_pred_lstm_p = predict_lstm(lstm_model, X_test.copy(), lstm_scaler, SEQUENCE_LENGTH)
    if len(y_pred_lstm_c) > 0:
        y_test_lstm_eval = y_test.iloc[len(y_test) - len(y_pred_lstm_c):]
        print("\nLSTM Evaluation:")
        print(f"Accuracy: {accuracy_score(y_test_lstm_eval, y_pred_lstm_c):.4f}")
    else: print("No LSTM predictions to evaluate due to sequence length vs test data size.")
except ValueError as e: print(f"LSTM Error: {e}")
except Exception as e: print(f"An unexpected error occurred with LSTM: {e}")

## 6. LightGBM Model

In [ ]:
print("\n--- Training LightGBM Model ---")
try:
    lgbm_model = train_lightgbm(X_train, y_train, params=LGBM_PARAMS)
    save_model(lgbm_model, LGBM_MODEL_FILENAME, models_dir=config.MODELS_DIR)
    y_pred_lgbm_c, y_pred_lgbm_p = predict_lightgbm(lgbm_model, X_test)
    print("\nLightGBM Evaluation:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred_lgbm_c):.4f}")
except Exception as e: print(f"An unexpected error occurred with LightGBM: {e}")

## 7. CatBoost Model

In [ ]:
print("\n--- Training CatBoost Model ---")
try:
    catboost_model = train_catboost(X_train, y_train, params=CATBOOST_PARAMS)
    save_model(catboost_model, CATBOOST_MODEL_FILENAME, models_dir=config.MODELS_DIR)
    y_pred_cat_c, y_pred_cat_p = predict_catboost(catboost_model, X_test)
    print("\nCatBoost Evaluation:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred_cat_c):.4f}")
except Exception as e: print(f"An unexpected error occurred with CatBoost: {e}")

## 8. CNN-LSTM Model

In [ ]:
print("\n--- Training CNN-LSTM Model ---")
CNNLSTM_MODEL_FILENAME = f"cnnlstm_model_{TICKER.lower()}.h5"
try:
    cnnlstm_model, cnnlstm_scaler = train_cnnlstm(
        X_train_df=X_train.copy(), y_train_series=y_train.copy(),
        sequence_length=SEQUENCE_LENGTH, **CNNLSTM_TRAIN_PARAMS
    )
    save_model(cnnlstm_model, CNNLSTM_MODEL_FILENAME, models_dir=config.MODELS_DIR)
    save_model(cnnlstm_scaler, f"cnnlstm_scaler_{TICKER.lower()}.pkl", models_dir=config.MODELS_DIR)
    
    y_pred_cnnlstm_c, y_pred_cnnlstm_p = predict_cnnlstm(cnnlstm_model, X_test.copy(), cnnlstm_scaler, SEQUENCE_LENGTH)
    if len(y_pred_cnnlstm_c) > 0:
        y_test_cnnlstm_eval = y_test.iloc[len(y_test) - len(y_pred_cnnlstm_c):]
        print("\nCNN-LSTM Evaluation:")
        print(f"Accuracy: {accuracy_score(y_test_cnnlstm_eval, y_pred_cnnlstm_c):.4f}")
        # print(classification_report(y_test_cnnlstm_eval, y_pred_cnnlstm_c, zero_division=0))
    else: print("No CNN-LSTM predictions to evaluate.")
except ValueError as e: print(f"CNN-LSTM Error: {e}")
except Exception as e: print(f"An unexpected error occurred with CNN-LSTM: {e}")

## 9. Transformer Model

In [ ]:
print("\n--- Training Transformer Model ---")
TRANSFORMER_MODEL_FILENAME = f"transformer_model_{TICKER.lower()}.h5"
TRANSFORMER_TRAIN_PARAMS['transformer_params']['embed_dim'] = X_train.shape[1] # Ensure embed_dim matches num_features
try:
    transformer_model, transformer_scaler = train_transformer(
        X_train_df=X_train.copy(), y_train_series=y_train.copy(),
        sequence_length=SEQUENCE_LENGTH, **TRANSFORMER_TRAIN_PARAMS
    )
    save_model(transformer_model, TRANSFORMER_MODEL_FILENAME, models_dir=config.MODELS_DIR)
    save_model(transformer_scaler, f"transformer_scaler_{TICKER.lower()}.pkl", models_dir=config.MODELS_DIR)
    
    y_pred_transformer_c, y_pred_transformer_p = predict_transformer(transformer_model, X_test.copy(), transformer_scaler, SEQUENCE_LENGTH)
    if len(y_pred_transformer_c) > 0:
        y_test_transformer_eval = y_test.iloc[len(y_test) - len(y_pred_transformer_c):]
        print("\nTransformer Evaluation:")
        print(f"Accuracy: {accuracy_score(y_test_transformer_eval, y_pred_transformer_c):.4f}")
        # print(classification_report(y_test_transformer_eval, y_pred_transformer_c, zero_division=0))
    else: print("No Transformer predictions to evaluate.")
except ValueError as e: print(f"Transformer Error: {e}")
except Exception as e: print(f"An unexpected error occurred with Transformer: {e}")

## 10. Prophet Model (as before)

In [ ]:
print("\n--- Training Prophet Model ---")
prophet_train_df = ohlcv_train[['Close']].reset_index()
prophet_train_df.rename(columns={'Date': 'ds', 'Close': 'y'}, inplace=True)
if 'ds' not in prophet_train_df.columns and isinstance(prophet_train_df.index, pd.DatetimeIndex):
    prophet_train_df.index.name = 'ds'
    prophet_train_df = prophet_train_df.reset_index()

if 'ds' in prophet_train_df.columns and 'y' in prophet_train_df.columns:
    try:
        prophet_model = train_prophet(prophet_train_df)
        save_model(prophet_model, PROPHET_MODEL_FILENAME, models_dir=config.MODELS_DIR)
        print("Prophet model trained and saved.")
        future_periods_for_test = len(ohlcv_test)
        prophet_forecast = predict_prophet(prophet_model, periods=future_periods_for_test, freq='B')
        # Prophet plotting and signal evaluation (simplified for brevity)
        # fig1 = prophet_model.plot(prophet_forecast)
        # plt.show()
    except Exception as e: print(f"Prophet processing error: {e}")
else: print("Prophet training data not prepared correctly.")

## 11. (Placeholder) Optuna Hyperparameter Tuning

In [ ]:
print("\nOptuna setup would be here.")